In [1]:
import torch
import torchvision
import numpy as np

## Numpy

In [2]:
##Initialize tensor
a = torch.ones(5)
print(a)


 1
 1
 1
 1
 1
[torch.FloatTensor of size 5]



In [3]:
#Convert tensor to numpy array
b = a.numpy()
print(b)

[ 1.  1.  1.  1.  1.]


In [4]:
#Convert numpy array to tensor
a = np.ones(5)
b = torch.from_numpy(a)
print(b)


 1
 1
 1
 1
 1
[torch.DoubleTensor of size 5]



In [9]:
# let us run this cell only if CUDA is available
x = torch.ones(5)
y = torch.ones(5)
if torch.cuda.is_available():
    x = x.cuda()
    y = y.cuda()
    x + y
    
def add():
    x + y
%timeit -n 1 add()

1 loop, best of 3: 5.56 µs per loop


A microsecond µs is 1 millionth of a second

## Autograd

In [20]:
from torch.autograd import Variable
from torch import Tensor

In [15]:
x = Variable(torch.ones(2,2), requires_grad=True)
print (x)
print (x.data)
print (x.creator)

Variable containing:
 1  1
 1  1
[torch.FloatTensor of size 2x2]


 1  1
 1  1
[torch.FloatTensor of size 2x2]

None


In [17]:
y = x + 2
print (y)
print (y.creator)

Variable containing:
 3  3
 3  3
[torch.FloatTensor of size 2x2]



In [28]:
z = y*y*3
out = z.mean()
out

Variable containing:
 27
[torch.FloatTensor of size 1]

In [29]:
out.backward()

In [30]:
print(x.grad)

Variable containing:
 4.5000  4.5000
 4.5000  4.5000
[torch.FloatTensor of size 2x2]



## Neural Networks

In [31]:
import torch.nn as nn
import torch.nn.functional as F

In [39]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution kernel
        self.conv1 = nn.Conv2d(1,6,5)
        self.conv2 = nn.Conv2d(6,16,5)
        self.fc1 = nn.Linear(16*5*5, 120) # an affine operation: y = Wx + b
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)
        
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2)) #2x2 pool window
        x = F.max_pool2d(F.relu(self.conv2(x)), 2) #if shape is square, you only need to define one number
        x = x.view(-1, self.num_flat_features(x)) #Flatten()
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    ## Backward() function is automatically defined for you!!
    
    def num_flat_features(self, x):
        #basically counting parameters and flattening 
        size = x.size()[1:] # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [40]:
net = NeuralNetwork()
print (net)

NeuralNetwork (
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear (400 -> 120)
  (fc2): Linear (120 -> 84)
  (fc3): Linear (84 -> 10)
)


In [43]:
X = Variable(torch.randn(1,1,32,32)) #nSamples x nChannels x Height x Width
yHat = net(X)
yHat

Variable containing:
-0.0449 -0.0343  0.1672  0.0041  0.1280 -0.0358  0.1536  0.1080 -0.0093  0.0831
[torch.FloatTensor of size 1x10]

## Loss Function

http://pytorch.org/docs/nn.html#loss-functions

In [52]:
yHat = net(X)
target = Variable(torch.range(1, 10))  # a dummy target, for example
criterion = nn.MSELoss()

loss = criterion(yHat, target)
print(loss)

Variable containing:
 37.8060
[torch.FloatTensor of size 1]



In [53]:
print(loss.creator)  # MSELoss
print(loss.creator.previous_functions[0][0])  # Linear
print(loss.creator.previous_functions[0][0].previous_functions[0][0])  # ReLU

## Backprop

In [54]:
#First clear the existing gradients!! 
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

#Backprop as simple as..
loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
Variable containing:
 0
 0
 0
 0
 0
 0
[torch.FloatTensor of size 6]

conv1.bias.grad after backward
Variable containing:
 0.1241
-0.0238
-0.2076
-0.0456
-0.0305
-0.1453
[torch.FloatTensor of size 6]



## Update Weights

In [55]:
#SGD
lr = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * lr)  #subtract gradient from weights...

In [57]:
#Other optimizers
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr=0.01)
criterion = nn.MSELoss()

In [58]:
#using the optimizers
#Zero the gradient
optimizer.zero_grad()
yHat = net(X)
loss = criterion(yHat,target)
loss.backward()
optimizer.step() #update the weights

## CNNs

https://chsasank.github.io/pytorch-tutorials/beginner/blitz/cifar10_tutorial.html